In [1]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from src.model import DogClassifier
from src.tools import MetricStorer, seed_everything
from src.config import DATASET_PATHS, CNFG, SAVE_METRICS_PATH, SAVE_WEIGHTS_PATH
from src.loader import WoofDataset, MultipleDataset, train_transforms, val_transforms

In [2]:
seed_everything(CNFG['seed'])

In [3]:
labels = pd.read_csv(DATASET_PATHS['labels'])
labels.sample(5)

,path,noisy_labels_0,noisy_labels_1,noisy_labels_5,noisy_labels_25,noisy_labels_50,is_valid
5687,train/n02099601/n02099601_1639.JPEG,n02099601,n02099601,n02099601,n02099601,n02099601,False
937,train/n02115641/n02115641_14534.JPEG,n02115641,n02115641,n02115641,n02115641,n02115641,False
7727,train/n02093754/n02093754_8005.JPEG,n02093754,n02093754,n02093754,n02093754,n02099601,False
7635,train/n02093754/n02093754_1536.JPEG,n02093754,n02093754,n02096294,n02093754,n02093754,False
12614,val/n02093754/n02093754_590.JPEG,n02093754,n02093754,n02093754,n02093754,n02093754,True


In [4]:
labels.isna().sum()

path               0
noisy_labels_0     0
noisy_labels_1     0
noisy_labels_5     0
noisy_labels_25    0
noisy_labels_50    0
is_valid           0
dtype: int64

In [5]:
LABEL_COL = 'noisy_labels_0'

In [6]:
labels = labels[['path', LABEL_COL]]

In [7]:
labels[LABEL_COL] = pd.Categorical(labels[LABEL_COL])

In [8]:
labels['codes'] = labels[LABEL_COL].cat.codes

In [9]:
labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12954 entries, 0 to 12953
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   path            12954 non-null  object  
 1   noisy_labels_0  12954 non-null  category
 2   codes           12954 non-null  int8    
dtypes: category(1), int8(1), object(1)
memory usage: 127.0+ KB


In [10]:
train_labels = labels[:9025]
val_labels = labels[9025:]

In [11]:
train_labels[LABEL_COL].value_counts()

n02093754    949
n02099601    949
n02096294    943
n02087394    942
n02086240    941
n02115641    940
n02088364    932
n02105641    928
n02111889    921
n02089973    580
Name: noisy_labels_0, dtype: int64

In [12]:
val_labels[LABEL_COL].value_counts()

n02111889    429
n02105641    422
n02088364    418
n02115641    410
n02086240    409
n02087394    408
n02096294    407
n02093754    401
n02099601    401
n02089973    224
Name: noisy_labels_0, dtype: int64

In [13]:
train_dataset = WoofDataset(files=train_labels['path'].to_list(),
                            labels=train_labels['codes'].to_list(),
                            data_path=DATASET_PATHS['main_path'],
                            transforms=train_transforms,
                            mode='train')

val_dataset = WoofDataset(files=val_labels['path'].to_list(),
                            labels=val_labels['codes'].to_list(),
                            data_path=DATASET_PATHS['main_path'],
                            transforms=val_transforms,
                            mode='val')

In [14]:
train_multi_dataset = MultipleDataset(train_dataset)
val_multi_dataset = MultipleDataset(val_dataset)

In [15]:
train_loader = DataLoader(train_multi_dataset, batch_size=CNFG['batch_size'], num_workers=CNFG['num_workers'])
val_loader = DataLoader(val_multi_dataset, batch_size=CNFG['batch_size'], num_workers=CNFG['num_workers'])

In [16]:
print(f'Train length - {len(train_multi_dataset)}, val length - {len(val_multi_dataset)}')

Train length - 9025, val length - 3929


In [17]:
model = DogClassifier(CNFG['model_arch'], 10, pretrained=True)
model.to(CNFG['device'])
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=CNFG['lr'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.3)

storer = MetricStorer(
    metrics=('acc'),
    raw_names=('train_preds', 'test_preds',
                'train_real', 'test_real'),
    folds=('train', 'test'),
    path=SAVE_METRICS_PATH,
    esr=CNFG['esr']
)

In [ ]:
for epoch in range(CNFG['epochs']):
    torch.cuda.empty_cache()
    model.train()
    train_loss = test_loss = 0
    print('LR - {}'.format([group['lr'] for group in optimizer.param_groups]))
    for img, target in train_loader:
        img, target = img.to(CNFG['device']), target.to(CNFG['device'])
        img = img.permute(0, 3, 1, 2)
        pred = model(img.float())
        loss = criterion(pred, target.long())
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), CNFG['grad_clip'])
        optimizer.step()
        storer.add_loss(loss, 'train')
        storer.apply_metric(name=('acc',),
                           pred=pred.argmax(dim=1),
                           real=target,
                           foldname='train')
        storer.add_raw('train_preds', storer.to_numpy(pred))
        storer.add_raw('train_real', storer.to_numpy(target))

    model.eval()
    torch.cuda.empty_cache()
    with torch.no_grad():
        for img, target in val_loader:
            img, target = img.to(CNFG['device']), target.to(CNFG['device'])
            img = img.permute(0, 3, 1, 2)
            pred = model(img.float())
            loss = criterion(pred, target.long())
            storer.add_loss(loss, 'test')
            storer.apply_metric(name=('acc',),
                           pred=pred.argmax(dim=1),
                           real=target,
                           foldname='test')
            storer.add_raw('test_preds', storer.to_numpy(pred))
            storer.add_raw('test_real', storer.to_numpy(target))
        scheduler.step(np.mean(storer.temp_metrics['test_loss']))
    torch.cuda.empty_cache()
    storer.new_epoch()
    log_line, to_break, is_best = storer.print_last(inplace=True)
    print('Break? {}\nBest score? {}'.format(to_break, is_best))
    if is_best:
        torch.save({
            'epoch': epoch,
            'loss': loss,
            'model': model.module.state_dict() if CNFG['multi_gpu'] else model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()
        }, SAVE_WEIGHTS_PATH)
        storer.move_best_raw()
        storer.dump()
    for key in ['train_preds', 'test_preds', 'train_real', 'test_real']:
        storer.clear_raw(key)
    if to_break:
        break
print('Training done.')

LR - [0.001]
